In [ ]:
import numpy as np
import cv2
import pandas as pd
import os
import pickle
from google.colab import drive
from google.colab.patches import cv2_imshow
from sklearn.model_selection import train_test_split

In [ ]:
# Load data from pickle file# Connect Google Drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/test_opencv/'
# Hiển thị tất cả thư mục hoặc file
os.listdir(path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['models',
 '1.jpg',
 '2.jpg',
 'Baitap27.ipynb',
 'baitap30.ipynb',
 'FaceImage-20241031T084935Z-001',
 'FaceImage',
 'data_face_features.pickle',
 '31c_data.ipynb',
 'Untitled0.ipynb',
 'Untitled1.ipynb']

In [ ]:
# Load data from pickle file
data = pickle.load(open("/content/drive/MyDrive/test_opencv/data_face_features.pickle", mode="rb"))

In [ ]:
x = np.array(data['data'])
y = np.array(data['label'])

print(x)
print(y)

[[[ 0.0196759   0.16772693  0.1007451  ... -0.03204957  0.11536875
    0.07170223]]

 [[ 0.00048526  0.16548792  0.09117572 ... -0.03187338  0.11723722
    0.09059   ]]

 [[ 0.04667978  0.16546842  0.00638199 ... -0.0629909   0.1310076
    0.09049864]]

 ...

 [[-0.00195631  0.0665765   0.08486877 ...  0.10612075 -0.03592141
    0.00421265]]

 [[ 0.05728974  0.17266382 -0.08290198 ... -0.01169536  0.03234326
    0.01378663]]

 [[ 0.0430265   0.09132998 -0.14244998 ...  0.08621342  0.07641552
    0.03136611]]]
['1721031618_TruongThanhLuan' '1721031618_TruongThanhLuan'
 '1721031618_TruongThanhLuan' ... '162000339_HuynhAnhDuy'
 '162000339_HuynhAnhDuy' '162000339_HuynhAnhDuy']


In [ ]:
x.shape

(4698, 1, 128)

In [ ]:
y.shape

(4698,)

In [ ]:
x = x.reshape(-1, 128)

In [ ]:
x.shape

(4698, 128)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.9, random_state=1)

In [ ]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((4228, 128), (470, 128), (4228,), (470,))

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score

In [ ]:
def get_report(model, x_train, y_train, x_test, y_test):
  y_pred_train = model.predict(x_train)
  y_pred_test = model.predict(x_test)

  # accuracy score
  acc_train = accuracy_score(y_train, y_pred_train)
  acc_test = accuracy_score(y_test, y_pred_test)

  # f1 score
  f1_score_train = f1_score(y_train, y_pred_train, average="macro")
  f1_score_test = f1_score(y_test, y_pred_test, average="macro")

  # print
  print('Accurency Train = %0.2f'%acc_train)
  print('Accurency Test = %0.2f'%acc_test)
  print('F1 Score Train = %0.2f'%f1_score_train)
  print('F1 Score Test = %0.2f'%f1_score_test)

In [ ]:
model_logistic = LogisticRegression()
model_logistic.fit(x_train, y_train) # training

LogisticRegression()

In [ ]:
get_report(model_logistic, x_train, y_train, x_test, y_test)

Accurency Train = 0.72
Accurency Test = 0.72
F1 Score Train = 0.52
F1 Score Test = 0.55


In [ ]:
model_svc = SVC(probability=True)
model_svc.fit(x_train, y_train)

SVC(probability=True)

In [ ]:
get_report(model_svc, x_train, y_train, x_test, y_test)

Accurency Train = 0.91
Accurency Test = 0.86
F1 Score Train = 0.83
F1 Score Test = 0.75


In [ ]:
model_rf = RandomForestClassifier(n_estimators=20,)
model_rf.fit(x_train, y_train)

RandomForestClassifier(n_estimators=20)

In [ ]:
get_report(model_rf, x_train, y_train, x_test, y_test)

Accurency Train = 1.00
Accurency Test = 0.79
F1 Score Train = 1.00
F1 Score Test = 0.65


In [ ]:
model_voting = VotingClassifier(estimators=[
    ('logistic', LogisticRegression()),
    ('svm', SVC(probability=True)),
    ('rf', RandomForestClassifier(n_estimators=20,)),
], voting='soft', weights=[2,3,1])

In [ ]:
model_voting.fit(x_train, y_train)

VotingClassifier(estimators=[('logistic', LogisticRegression()),
                             ('svm', SVC(probability=True)),
                             ('rf', RandomForestClassifier(n_estimators=20))],
                 voting='soft', weights=[2, 3, 1])

In [ ]:
get_report(model_voting, x_train, y_train, x_test, y_test)

Accurency Train = 0.98
Accurency Test = 0.86
F1 Score Train = 0.97
F1 Score Test = 0.76


In [ ]:
# 3. Parameter Tuning
from sklearn.model_selection import GridSearchCV

model_grid = GridSearchCV(model_voting, param_grid={
    'svm__C': [3,5,7,10],
    'svm__gamma': [0.1, 0.3, 0.5],
    'rf__n_estimators': [5, 10,20],
    'rf__max_depth': [3, 5, 7],
    'voting': ['soft', 'hard']
}, scoring='accuracy', cv=3, n_jobs=1, verbose=2)

In [ ]:
model_grid.fit(x_train, y_train)

Fitting 3 folds for each of 216 candidates, totalling 648 fits
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   8.5s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   8.8s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   6.9s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   8.7s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   6.2s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   9.1s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   6.3s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   9.1s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   7.3s
[

GridSearchCV(cv=3,
             estimator=VotingClassifier(estimators=[('logistic',
                                                     LogisticRegression()),
                                                    ('svm',
                                                     SVC(probability=True)),
                                                    ('rf',
                                                     RandomForestClassifier(n_estimators=20))],
                                        voting='soft', weights=[2, 3, 1]),
             n_jobs=1,
             param_grid={'rf__max_depth': [3, 5, 7],
                         'rf__n_estimators': [5, 10, 20],
                         'svm__C': [3, 5, 7, 10], 'svm__gamma': [0.1, 0.3, 0.5],
                         'voting': ['soft', 'hard']},
             scoring='accuracy', verbose=2)

In [ ]:
model_best_estimator = model_grid.best_estimator_

In [ ]:
model_grid.best_score_

0.8479217525968655

In [ ]:
pickle.dump(model_best_estimator, open("/content/drive/MyDrive/test_opencv/ml_face_person_identity.pkl", mode='wb'))